# 🧪 Esempio di Modello con Hugging Face

## 📌 Obiettivo

Esplorare l’**architettura di un transformer reale** e generare testo tramite **Hugging Face Transformers**, comprendendo ogni fase del processo:

---

## 🧰 Setup Iniziale

1. **Caricamento del modello**:
   - Si utilizza il modello `Phi-3-mini` e il suo **tokenizer**.
   - L'esecuzione è su **CPU**, quindi è normale vedere warning legati alle performance.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto", 
    trust_remote_code=True,
    )

c:\Users\felip\Desktop\How Transformer LLMs Work\notebooks-codes\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\felip\Desktop\How Transformer LLMs Work\notebooks-codes\.venv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██

**Parametri di generazione**:
   - `max_new_tokens=50`: genera al massimo 50 token.
   - `do_sample=False`: attiva **greedy decoding** (equivalente a `temperature=0`).

In [2]:
# Create a pipeline 
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50, # ogni volta che diamo un prompt vogliamo che si generino al massimo 50 token
    do_sample=False # significa che stiamo usando il greedy decoding, ciascun token avrà la probabilità più alta di essere selezionato (come assegnare temperature=0)
)

## 🧪 Esecuzione: Prompt + Output

**Prompt di esempio**:
```text
Write an email apologizing to Sarah for the tragic gardening mishap.
```

Il modello genera un'email fino al 50° token. Puoi cambiare il prompt e osservare come il modello si comporta.

🖥️ L'esecuzione su CPU richiede più tempo (~2 min).
IL quale è il motivo per cui nell'industria è più comune l'uso di GPU dove i modelli vengono altamente ottimizzati, e molti di questi medotodi di efficienza che abbiamo discusso sono importanti per velocizzare le generazioni.

In [3]:
prompt = "Write an email apoligizing to Sarah for the traguc gardering mishap. Explain how it happened."

output = generator(prompt)

print(output[0]["generated_text"])

You are not running the flash-attention implementation, expect numerical differences.




Sarah,

I am deeply sorry for the unfortunate incident that occurred in your garden. I understand how much effort and love you put into maintaining it, and I feel terrible for the damage that was caused.



## 🧱 Anatomia del Modello

Stampando `model` si osserva la **architettura** del modello:

- **Tipo**: modello autoregressivo (causal language modeling), ciò significa che lo step di attention considera solo i token precedenti.
- **Architettura**:
  - 32 transformer **decoder layers**
  - **Dimensionalità vettoriale**: 3.072
  - **Feedforward dimension**: 16.000
  - **Vocabolario**: 32.064 token
  - **Componenti**:
    - Self-attention
    - Rotary embeddings
    - MLP (Multi-Layer Perceptron ovvero Feedforward network)
    - Activation + LayerNorm
    - Language modeling head (la fine del modello che prende in input il vettore finale di dimensioni 3.072 e da in output uno score per ciascuno dei token che il modello ha nel suo vocabolario ovvero 32.064 scores)

In [4]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

In [5]:
model.model

Phi3Model(
  (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
  (embed_dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-31): 32 x Phi3DecoderLayer(
      (self_attn): Phi3Attention(
        (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        (rotary_emb): Phi3RotaryEmbedding()
      )
      (mlp): Phi3MLP(
        (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
        (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
        (activation_fn): SiLU()
      )
      (input_layernorm): Phi3RMSNorm()
      (resid_attn_dropout): Dropout(p=0.0, inplace=False)
      (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      (post_attention_layernorm): Phi3RMSNorm()
    )
  )
  (norm): Phi3RMSNorm()
)

In [6]:
# accediamo al primo transformer block
model.model.layers[0]

Phi3DecoderLayer(
  (self_attn): Phi3Attention(
    (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
    (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (mlp): Phi3MLP(
    (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
    (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
    (activation_fn): SiLU()
  )
  (input_layernorm): Phi3RMSNorm()
  (resid_attn_dropout): Dropout(p=0.0, inplace=False)
  (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
  (post_attention_layernorm): Phi3RMSNorm()
)

## ⚙️ Tokenizzazione e Decoding

1. **Tokenizzazione**:

In [7]:
prompt = "The capital of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids

tensor([[ 450, 7483,  310, 3444,  338]])

2. **Passaggio attraverso il modello (senza head)**:

Notare che inviamo gli ids al componente `model` del modello.
Quindi questo non passa attraverso la head di language modeling.
Fluisce attraverso lo stack dei transformer blocks.

![alt text]("../images/compo.png")

Vediamo che l'otput è un tensor di dimensione `(1, num_tokens, 3072)`

Dove 1 indica il numero di batch, abbiamo passato al modello solo un testo, invece surante l'addestramento si hanno molti batch che contentono molti testi.

5 è il numero di token che abbiamo passato al modello, mentre 3072 è la dimensione dei vettori di output.

In [8]:
# inviamo gli inputs_isd al modello
model_output = model.model(input_ids)
model_output[0].shape

torch.Size([1, 5, 3072])

3. **Passaggio alla head di language modeling**:

Prende in input vettori di dimensione 3072 e da in output gli score per ciascuno dei token del vocabolario del tokenizer ovvero 32064.

Il token con lo score/probabilità più alto, dei 32064 sarà il token che si da in output.

In [9]:
lm_head_output = model.lm_head(model_output[0])
lm_head_output.shape

torch.Size([1, 5, 32064])

4. **Predizione del prossimo token**:

Per ottenere il primo token di output del modello per questo prompt prendiamo l'ultimo token della sequenza, che sarà il token di output.

In [10]:
token_id = lm_head_output[0, -1]
token_id

tensor([27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000],
       dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [12]:
token_id.shape

torch.Size([32064])

token_id = lm_head_output[0, -1].argmax(-1)

lm_head_output[0, -1] prende l'ultima posizione della sequenza (-1) nel primo esempio del batch (0).

È un vettore di logits di dimensione vocab_size.

argmax(-1) prende l’indice (cioè l’ID del token) con il logit più alto ⇒ il token che il modello sceglierebbe se stesse facendo greedy decoding.

✅ Quindi, questa riga seleziona il token che il modello vuole generare dopo l’ultima posizione della sequenza.

✅ Quindi… cosa fa argmax(-1)?
-1 è l’ultima dimensione del tensore, cioè:

Se hai un tensore 2D di shape (sequence_length, vocab_size),
allora argmax(-1) cerca il valore massimo per ogni riga (cioè per ogni posizione nella sequenza) lungo la dimensione dei token del vocabolario.
Se il tensore è 1D trova il valore massimo del vettore.

In [13]:
token_id = token_id.argmax(-1)
# token con la probabilità più alta
token_id

tensor(3681)

✔️ Il token predetto per "The capital of France is" → `"Paris"`

In [14]:
# decodifichiamo tale token_id 
tokenizer.decode(token_id)

'Paris'

## 📚 Conclusioni

Il modello non vede mai il testo dato in input (prompt). 
Il modello vede solamente questa lista di numeri (token_ids), e da in output altre liste di numeri (ids dei token previsti che vengono decodificati col tokenizer). 

Tutto ciò che il modello tocca sono solo gli ids dei token e i loro embeddings. Non vede alcuna lettera del testo che diamo in input.

Questo mostra quanto affascinante è il modo in cui i language models operano senza mai vedere veramente il linguaggio umano.

Questo esempio dimostra:
- Come funziona la **generazione autoregressiva**
- Come **token**, **matrici**, e **proiezioni** si integrano nei Transformer
- Come accedere al modello per ispezionarlo in profondità